#  MLOps Manual to Repeatable Workflow

<div class="alert alert-warning"> 
	⚠️ <strong> PRE-REQUISITE: </strong> Before proceeding with this notebook, please ensure that you have executed the <code>1-data-prep-feature-store.ipynb</code> and <code>2-training-registry.ipynb</code> Notebooks</li>
</div>

## Contents

- [Introduction](#Introduction)
- [SageMaker Endpoint](#SageMaker-Endpoint)

## Introduction

This is our third notebook which will explore the model deployment of ML workflow.

Here, we will put on the hat of a `Data Scientist` and will perform the task of model deployment which includes fetching the right model and deploying it for inference.  

For this task we will be using Amazon SageMaker Model Hosting capabilities.

Let's get started!

**Imports**

In [ ]:
%store -r

In [ ]:
from urllib.parse import urlparse
import io
from sagemaker.model import ModelPackage
import boto3
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import numpy as np
import pathlib
from sagemaker.feature_store.feature_group import FeatureGroup

**Session variables**

In [ ]:
# Useful SageMaker variables
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role_arn= sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
s3_client = boto3.client('s3', region_name=region)
sagemaker_client = boto3.client('sagemaker')

## SageMaker Endpoint

You can also deploy your trained model as [SageMaker hosted endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html) which serves real-time predictions from a trained model. The endpoint will retrieve the model created during training and deploy it within a SageMaker scikit-learn container. This all can be accomplished with one line of code. Note that it will take several minutes to deploy the model to a hosted endpoint.

Let's get the model we registered in the Model Registry.

In [ ]:
xgboost_regressor_model = ModelPackage(
    role_arn,
    model_package_arn=model_package_arn,
    name=model_name
)

It's current status is `PendingApproval`. In order to use this model for offline predictions or as a real-time endpoint, we'll need to update its status to `Approved`.

In [ ]:
sagemaker_client.update_model_package(
    ModelPackageArn=xgboost_regressor_model.model_package_arn,
    ModelApprovalStatus='Approved'
)

Now we can deploy it!

In [ ]:
xgboost_regressor_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=f'{model_name}-endpoint'
)

Let's test this real-time endpoint by passing it some data and getting a real-time prediction back.

In [ ]:
# Read in test set that was used for batch transform
fs_group = FeatureGroup(name=test_feature_group_name, sagemaker_session=sagemaker_session)  
query = fs_group.athena_query()
table = query.table_name
query_string = f'SELECT {features_to_select} FROM "sagemaker_featurestore"."{table}"  ORDER BY record_id'
query_results= 'sagemaker-featurestore'
output_location = f's3://{bucket}/{query_results}/query_results/'
query.run(query_string=query_string, output_location=output_location)
query.wait()
df = query.as_dataframe()
df.head()

In [ ]:
# Attach to the SageMaker endpoint
predictor = Predictor(endpoint_name=f'{model_name}-endpoint',
                      sagemaker_session=sagemaker_session,
                      serializer=CSVSerializer(),
                      deserializer=JSONDeserializer())

# Get a real-time prediction
predictor.predict(df.drop(columns=["price"]).to_csv(index=False, header=False))